# Panoptic segmentation using DETR

In this notebook we demonstrate how to explore the panoptic segmentation capabilities of DETR. The prediction occurs in several steps: 

1.   The model predicts a box and a binary mask for each object queries
2.   We filter the predictions for which the confidence is < 85%
3.   Finally, the remaining masks are merged together using a pixel-wise argmax

For simplicity, we rely on DETR's postprocessor to execute 2 and 3. We encourage to take a look at the corresponding code to get a better understanding of the process.

Finally, we visualize the final prediction using Detectron2.

## Preliminaries

This section contains the necessary boiler-plate. Run it first.

In [ ]:
from PIL import Image
import requests
import io
import math
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import torch
from torch import nn
from torchvision.models import resnet50
import torchvision.transforms as T
import numpy
torch.set_grad_enabled(False);

Mount Google Drive

In [ ]:
# Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The panoptic API is required for panoptic inference

In [ ]:
! pip install git+https://github.com/cocodataset/panopticapi.git


  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-req-build-qsxdyc6c
  Running command git clone -q https://github.com/cocodataset/panopticapi.git /tmp/pip-req-build-qsxdyc6c
  Created wheel for panopticapi: filename=panopticapi-0.1-py3-none-any.whl size=8306 sha256=a85ef6660c9219b2da2472c4105141a537f79900282c623b7d4b3cb32f3abedc
  Stored in directory: /tmp/pip-ephem-wheel-cache-qey01lt5/wheels/ad/89/b8/b66cce9246af3d71d65d72c85ab993fd28e7578e1b0ed197f1
Successfully built panopticapi


In [ ]:
import panopticapi
from panopticapi.utils import id2rgb, rgb2id

In [ ]:
# Detectron2 uses a different numbering scheme, we build a conversion table
# coco2d2 = {}
# count = 0
# for i, c in enumerate(CLASSES):
#   if c != "N/A":
#     coco2d2[i] = count
#     count+=1

# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])



## Using a model from hub

We load a pre-trained model directly from torch hub. Note that we also request the post-processor

In [ ]:
model, postprocessor = torch.hub.load('facebookresearch/detr', 'detr_resnet101_panoptic', pretrained=True, return_postprocessor=True, num_classes=250)
model.eval();

Downloading: "https://github.com/facebookresearch/detr/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

Downloading: "https://dl.fbaipublicfiles.com/detr/detr-r101-panoptic-40021d53.pth" to /root/.cache/torch/hub/checkpoints/detr-r101-panoptic-40021d53.pth


  0%|          | 0.00/237M [00:00<?, ?B/s]

This returns a mask for each query, let us visualize the high confidence ones

In [ ]:
def get_box_coords(boxes):
    x, y, w, h = boxes.unbind(1)
    x0, y0 = (x - 0.5 * w), (y - 0.5 * h)
    x1, y1 = (x + 0.5 * w), (y + 0.5 * h)
    box = [x0, y0, x1, y1]
    return torch.stack(box, dim=1)

# Scale box from [0,1]x[0,1] to [0, width]x[0, height]
def scale_boxes(output_box, width, height):
    box_coords = get_box_coords(output_box)
    scale_tensor = torch.Tensor(
                 [width, height, width, height])
    return box_coords * scale_tensor

Create a Data Frame to store the Predicted Bounding Boxes and Segmentation Masks 

In [ ]:
import pandas as pd
import os
import itertools
import seaborn as sns
#df=pd.DataFrame(columns= ['dir','file_name','segmentation','area,bbox','category_id','is_thing','panoptic_seg_id','panoptic_seg_shape'])

Next, we retrieve an image on which we wish to test the model. Here, we use an image from the validation set of COCO

In [ ]:
# from PIL import Image

# img_name = '/content/img_104.png'
# im = Image.open(img_name).convert('RGB')


# Prepare the Dataframe 

In [ ]:
'/content/drive/MyDrive/pipe_fittings/images',
'/content/drive/MyDrive/rcc_hume_pipes/images',
'/content/drive/MyDrive/refrigerant_gas/images',
'/content/drive/MyDrive/river_sand/images',
'/content/drive/MyDrive/rmc_batching_plant/images',
'/content/drive/MyDrive/rmu_units/images',
'/content/drive/MyDrive/sanitary_fixtures/images',
'/content/drive/MyDrive/skid_steer_loader_(bobcat)/images',
'/content/drive/MyDrive/smoke_detectors/images',
'/content/drive/MyDrive/split_units/images',
'/content/drive/MyDrive/structural_steel_-_channel/images',
'/content/drive/MyDrive/switch_boards_and_switches/images',
'/content/drive/MyDrive/texture_paint/images',
'/content/drive/MyDrive/threaded_rod/images',
'/content/drive/MyDrive/transit_mixer/images',
'/content/drive/MyDrive/vcb_panel/images',
'/content/drive/MyDrive/vitrified_tiles/images',
'/content/drive/MyDrive/vrf_units/images',
'/content/drive/MyDrive/water_tank/images',
'/content/drive/MyDrive/wheel_loader/images',
'/content/drive/MyDrive/wood_primer/images'

In [ ]:
dir_list=['/content/drive/MyDrive/vitrified_tiles/images',
'/content/drive/MyDrive/vrf_units/images',
'/content/drive/MyDrive/water_tank/images',
'/content/drive/MyDrive/wheel_loader/images',
'/content/drive/MyDrive/wood_primer/images']
for dir in dir_list:
  print('Currently Working on Directory:', dir)
  os.chdir(dir)
  df=pd.DataFrame(columns=['dir','file_name','segmentation','area','bbox','category_id','is_thing','panoptic_seg_id','panoptic_seg_shape'])
  for file in (os.listdir(dir)):
    im = Image.open(file)
    if im.mode == 'RGB':
      # mean-std normalize the input image (batch-size: 1)
      img = transform(im).unsqueeze(0)
      out = model(img)
      # compute the scores, excluding the "no-object" class (the last one)
      scores = out["pred_logits"].softmax(-1)[..., :-1].max(-1)[0]
      # threshold the confidence
      keep = scores > 0.85
      # convert boxes from [0; 1] to image scales
      width = im.size[0]
      height = im.size[1]
      bboxes_scaled = scale_boxes(out['pred_boxes'][keep], width, height)
      seg= out["pred_masks"][keep]
      # the post-processor expects as input the target size of the predictions (which we set here to the image size)
      result = postprocessor(out, torch.as_tensor(img.shape[-2:]).unsqueeze(0))[0]
      
      
      palette = itertools.cycle(sns.color_palette())
      # The segmentation is stored in a special-format png
      panoptic_seg = Image.open(io.BytesIO(result['png_string']))
      panoptic_seg = numpy.array(panoptic_seg, dtype=numpy.uint8).copy()
      # We retrieve the ids corresponding to each mask
      panoptic_seg_id = rgb2id(panoptic_seg)
      #print(len(bboxes_scaled), len(panoptic_seg),len(panoptic_seg[0]))
      for i in range(len(result['segments_info'])):
        df = df.append({'dir':dir,'file_name':file,'segmentation':seg[i][0],'area':result['segments_info'][i]['area'],'bbox':bboxes_scaled[i],'category_id':result['segments_info'][i]['category_id'],'is_thing':result['segments_info'][i]['isthing'],'panoptic_seg_id':panoptic_seg_id[i],'panoptic_seg_shape':panoptic_seg[i]},ignore_index=True)
        #print(panoptic_seg_id[i],panoptic_seg[i])
  df.to_csv('output.csv')
  print('CSV written successfully')



Currently Working on Directory: /content/drive/MyDrive/vitrified_tiles/images


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/usr/local/lib/python3.7/dist-packages/torch/nn/fun

CSV written successfully
Currently Working on Directory: /content/drive/MyDrive/vrf_units/images
CSV written successfully
Currently Working on Directory: /content/drive/MyDrive/water_tank/images
CSV written successfully
Currently Working on Directory: /content/drive/MyDrive/wheel_loader/images
CSV written successfully
Currently Working on Directory: /content/drive/MyDrive/wood_primer/images
CSV written successfully
